### This Jupyter Notebook will take the Master dataframe and add some columns for the machine learning
1 - Calculate the Quantile of the forward one-month return, which will be the forecasted item for ML

2 - Calculate 3 Momentum factors, prior-12-month, 3-month and 1-month returns, three more inputs for ML

In [1]:
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

import sqlalchemy
from sqlalchemy.types import Integer, Text, String, Float, Date
from sqlalchemy import create_engine

In [2]:
# Connect to the SQLite database
database_name = "../data/CompanyData.sqlite"
engine = create_engine(f"sqlite:///{database_name}", echo=False)

In [3]:
# Read the existing MasterData file into a dataframe
master_df = pd.read_sql_query("SELECT * FROM MasterData", engine)
master_df.head(12)

,monthend_date,ticker,price,wealth_index,end_wealth_index,total_return,cash,st_debt,lt_debt,equity,...,market_cap,net_debt,net_debt_capital,enterprise_value,ev_revenue,revenue_ev,ev_ebit,ebit_ev,name,sector
0,2014-12-31,A,40.94,39.0850,36.1469,-7.517206,2.218000e+09,0.0,1.663000e+09,5.301000e+09,...,1.362185e+10,-555000000.0,-11.694058,1.306685e+10,3.227978,0.309791,31.185810,0.032066,Agilent Technologies Inc,Health Care
1,2015-01-31,A,37.77,36.1469,40.3961,11.755365,2.218000e+09,0.0,1.663000e+09,5.301000e+09,...,1.256711e+10,-555000000.0,-11.694058,1.201211e+10,2.967418,0.336993,28.668518,0.034881,Agilent Technologies Inc,Health Care
2,2015-02-28,A,42.21,40.3961,39.8624,-1.321167,2.218000e+09,0.0,1.663000e+09,5.301000e+09,...,1.404442e+10,-555000000.0,-11.694058,1.348942e+10,3.332366,0.300087,32.194315,0.031061,Agilent Technologies Inc,Health Care
3,2015-03-31,A,41.55,39.8624,39.6897,-0.433240,2.118000e+09,0.0,1.658000e+09,4.237000e+09,...,1.386108e+10,-460000000.0,-12.178978,1.340108e+10,3.295888,0.303408,32.685561,0.030595,Agilent Technologies Inc,Health Care
4,2015-04-30,A,41.37,39.6897,39.5170,-0.435125,2.118000e+09,0.0,1.658000e+09,4.237000e+09,...,1.380103e+10,-460000000.0,-12.178978,1.334103e+10,3.281120,0.304774,32.539102,0.030732,Agilent Technologies Inc,Health Care
5,2015-05-31,A,41.19,39.5170,37.1057,-6.101931,2.118000e+09,0.0,1.658000e+09,4.237000e+09,...,1.374098e+10,-460000000.0,-12.178978,1.328098e+10,3.266351,0.306152,32.392644,0.030871,Agilent Technologies Inc,Health Care
6,2015-06-30,A,38.58,37.1057,39.3851,6.142992,2.197000e+09,0.0,1.656000e+09,4.158000e+09,...,1.280155e+10,-541000000.0,-14.957147,1.226055e+10,3.034037,0.329594,28.984741,0.034501,Agilent Technologies Inc,Health Care
7,2015-07-31,A,40.95,39.3851,34.9224,-11.330935,2.197000e+09,0.0,1.656000e+09,4.158000e+09,...,1.358795e+10,-541000000.0,-14.957147,1.304695e+10,3.228645,0.309727,30.843864,0.032421,Agilent Technologies Inc,Health Care
8,2015-08-31,A,36.31,34.9224,33.1139,-5.178625,2.197000e+09,0.0,1.656000e+09,4.158000e+09,...,1.204832e+10,-541000000.0,-14.957147,1.150732e+10,2.847641,0.351168,27.204062,0.036759,Agilent Technologies Inc,Health Care
9,2015-09-30,A,34.33,33.1139,36.4224,9.991273,2.075000e+09,0.0,1.655000e+09,4.103000e+09,...,1.140866e+10,-420000000.0,-11.403747,1.098866e+10,2.715931,0.368198,25.203341,0.039677,Agilent Technologies Inc,Health Care


In [4]:
# Calculate the quantile of total return, with Quantile 1 = highest return, Quantile 'n' = lowest return
# In this run, we are creating 3 groups, thus creating Tertiles
master_df['quantile_total_return'] = master_df.groupby('monthend_date')['total_return'] \
        .transform (lambda x: pd.qcut(x,3,labels=range(3,0,-1)))

In [5]:
# Define a function to add "n" months to a date
def add_months (start_date, n_months):
    
    # Parse the text string to get the current year and month, then add n_months
    temp_year = int(start_date[0:4])
    temp_month = int(start_date[5:7]) + n_months
    
    # If the 'temp_month' is greater than 12, then figure out how many years need to be added, and reset the month
    if (temp_month > 12):
        temp_year = temp_year + int(temp_month/12)
        temp_month = temp_month % 12
    
    if (temp_month == 0):
        new_date = datetime.datetime (temp_year-1, 12, 31) 
    elif (temp_month == 12):
        new_date = datetime.datetime (temp_year, 12, 31) 
    else:
        new_date = datetime.datetime (temp_year, temp_month + 1, 1) - relativedelta(days=1)
    return new_date.strftime("%Y-%m-%d")

In [6]:
# Calculate the trailing 1-month price returns
# Create a new dataframe with monthend_date and price for each ticker
prices_df = master_df[['monthend_date', 'ticker', 'price']].copy()

# This will become the price the beginning of the 1-month period, so change the name of price
prices_df.rename(columns={'price': 'price_1_month_ago'}, inplace=True)

# Since we are calculating a 1-month return, the price_1_month_ago matches up with the 1 month in the future
# Add 1 month to the monthend_date
prices_df['ending_date'] = [add_months (i,1) for i in prices_df['monthend_date']]

# Now, delete the column named "monthend_date"
prices_df.drop(columns=['monthend_date'], inplace=True)

# Change the name of "ending_date" to "monthend_date"
prices_df.rename(columns={'ending_date': 'monthend_date'}, inplace=True)

# Now, do an inner join of "prices_df" and "master_df" on "ticker" and "monthend_date"
prices2_df = pd.merge(prices_df, master_df,  how='inner', left_on=['ticker','monthend_date'], right_on = ['ticker','monthend_date'])

# Calculate the trailing 1-month return
prices2_df['trailing_1_month_return'] = (prices2_df['price'] / prices2_df['price_1_month_ago'] - 1) * 100

# Copy the ending dataframe to 'master'
master_df = prices2_df

In [7]:
# Calculate the trailing 3-month price returns
# Create a new dataframe with monthend_date and price for each ticker
prices_df = master_df[['monthend_date', 'ticker', 'price']].copy()

# This will become the price the beginning of the 3-month period, so change the name of price
prices_df.rename(columns={'price': 'price_3_months_ago'}, inplace=True)

# Since we are calculating a 3-month return, the price_3_months_ago matches up with the 3 months in the future
# Add 3 months to the monthend_date
prices_df['ending_date'] = [add_months (i,3) for i in prices_df['monthend_date']]

# Now, delete the column named "monthend_date"
prices_df.drop(columns=['monthend_date'], inplace=True)

# Change the name of "ending_date" to "monthend_date"
prices_df.rename(columns={'ending_date': 'monthend_date'}, inplace=True)

# Now, do an inner join of "prices_df" and "master_df" on "ticker" and "monthend_date"
prices2_df = pd.merge(prices_df, master_df,  how='inner', left_on=['ticker','monthend_date'], right_on = ['ticker','monthend_date'])

# Calculate the trailing 3-month return
prices2_df['trailing_3_month_return'] = (prices2_df['price'] / prices2_df['price_3_months_ago'] - 1) * 100

# Copy the ending dataframe to 'master'
master_df = prices2_df

In [8]:
# Calculate the trailing 12-month price returns
# Create a new dataframe with monthend_date and price for each ticker
prices_df = master_df[['monthend_date', 'ticker', 'price']].copy()

# This will become the price the beginning of the 12-month period, so change the name of price
prices_df.rename(columns={'price': 'price_12_months_ago'}, inplace=True)

# Since we are calculating a 12-month return, the price_12_months_ago matches up with the 12 months in the future
# Add 12 months to the monthend_date
prices_df['ending_date'] = [add_months (i,12) for i in prices_df['monthend_date']]

# Now, delete the column named "monthend_date"
prices_df.drop(columns=['monthend_date'], inplace=True)

# Change the name of "ending_date" to "monthend_date"
prices_df.rename(columns={'ending_date': 'monthend_date'}, inplace=True)

# Now, do an inner join of "prices_df" and "master_df" on "ticker" and "monthend_date"
prices2_df = pd.merge(prices_df, master_df,  how='inner', left_on=['ticker','monthend_date'], right_on = ['ticker','monthend_date'])

# Calculate the trailing 12-month return
prices2_df['trailing_12_month_return'] = (prices2_df['price'] / prices2_df['price_12_months_ago'] - 1) * 100

# Copy the ending dataframe to 'master'
master_df = prices2_df

In [9]:
# Write the calculations to a CSV file, to make sure it worked as expected
test_df = master_df[['monthend_date','ticker','price','price_1_month_ago','price_3_months_ago','price_12_months_ago', \
                    'trailing_1_month_return','trailing_3_month_return','trailing_12_month_return']]
test_df.to_csv('test_df.csv')

In [10]:
# Check to see how much data remains, by monthend_date
df = master_df.groupby('monthend_date').size()
df.head(100)

monthend_date
2016-02-29     15
2016-03-31    429
2016-04-30    467
2016-05-31    472
2016-06-30    473
2016-07-31    473
2016-08-31    474
2016-09-30    474
2016-10-31    474
2016-11-30    474
2016-12-31    477
2017-01-31    477
2017-02-28    477
2017-03-31    478
2017-04-30    478
2017-05-31    478
2017-06-30    478
2017-07-31    478
2017-08-31    478
2017-09-30    478
2017-10-31    478
2017-11-30    478
2017-12-31    479
2018-01-31    479
2018-02-28    479
2018-03-31    480
2018-04-30    481
2018-05-31    481
2018-06-30    483
2018-07-31    483
2018-08-31    483
2018-09-30    484
2018-10-31    484
2018-11-30    484
2018-12-31    484
2019-01-31    484
2019-02-28    484
2019-03-31    485
2019-04-30    486
2019-05-31    486
2019-06-30    487
2019-07-31    487
2019-08-31    487
dtype: int64

In [11]:
# Drop 2/29/2016 as there is not enough information 
master_df = master_df[master_df.monthend_date != '2016-02-29']
df = master_df.groupby('monthend_date').size()
df.head(100)

monthend_date
2016-03-31    429
2016-04-30    467
2016-05-31    472
2016-06-30    473
2016-07-31    473
2016-08-31    474
2016-09-30    474
2016-10-31    474
2016-11-30    474
2016-12-31    477
2017-01-31    477
2017-02-28    477
2017-03-31    478
2017-04-30    478
2017-05-31    478
2017-06-30    478
2017-07-31    478
2017-08-31    478
2017-09-30    478
2017-10-31    478
2017-11-30    478
2017-12-31    479
2018-01-31    479
2018-02-28    479
2018-03-31    480
2018-04-30    481
2018-05-31    481
2018-06-30    483
2018-07-31    483
2018-08-31    483
2018-09-30    484
2018-10-31    484
2018-11-30    484
2018-12-31    484
2019-01-31    484
2019-02-28    484
2019-03-31    485
2019-04-30    486
2019-05-31    486
2019-06-30    487
2019-07-31    487
2019-08-31    487
dtype: int64

In [12]:
cols = master_df.columns.tolist()
cols

['ticker',
 'price_12_months_ago',
 'monthend_date',
 'price_3_months_ago',
 'price_1_month_ago',
 'price',
 'wealth_index',
 'end_wealth_index',
 'total_return',
 'cash',
 'st_debt',
 'lt_debt',
 'equity',
 'revenue',
 'ebit',
 'net_income',
 'basic_eps',
 'diluted_eps',
 'shares',
 'qtr_date',
 'price_earnings',
 'earnings_yield',
 'book_value',
 'price_book',
 'book_yield',
 'market_cap',
 'net_debt',
 'net_debt_capital',
 'enterprise_value',
 'ev_revenue',
 'revenue_ev',
 'ev_ebit',
 'ebit_ev',
 'name',
 'sector',
 'quantile_total_return',
 'trailing_1_month_return',
 'trailing_3_month_return',
 'trailing_12_month_return']

In [13]:
new_cols = ['monthend_date', 'ticker', 'price', 'wealth_index', 'end_wealth_index', 'total_return',
            'cash', 'st_debt', 'lt_debt', 'equity', 'revenue', 'ebit', 'net_income', 'basic_eps', 'diluted_eps',
            'shares', 'qtr_date', 'price_earnings', 'earnings_yield', 'book_value', 'price_book', 'book_yield',
            'market_cap', 'net_debt', 'net_debt_capital', 'enterprise_value', 'ev_revenue', 'revenue_ev', 'ev_ebit',
            'ebit_ev', 'name', 'sector', 'quantile_total_return', 'price_1_month_ago','price_3_months_ago',
            'price_12_months_ago', 'trailing_1_month_return', 'trailing_3_month_return', 'trailing_12_month_return']
master_df = master_df[new_cols]
master_df.head()

,monthend_date,ticker,price,wealth_index,end_wealth_index,total_return,cash,st_debt,lt_debt,equity,...,ebit_ev,name,sector,quantile_total_return,price_1_month_ago,price_3_months_ago,price_12_months_ago,trailing_1_month_return,trailing_3_month_return,trailing_12_month_return
0,2016-04-30,A,40.92,39.6930,44.5140,12.145718,1.931000e+09,80000000.0,1.653000e+09,4.045000e+09,...,0.041902,Agilent Technologies Inc,Health Care,1.0,39.85,37.65,41.37,2.685069,8.685259,-1.087745
1,2016-05-31,A,45.89,44.5140,43.1432,-3.079481,1.931000e+09,80000000.0,1.653000e+09,4.045000e+09,...,0.037304,Agilent Technologies Inc,Health Care,3.0,40.92,37.35,41.19,12.145650,22.864793,11.410537
2,2016-06-30,A,44.36,43.1432,46.7904,8.453708,2.139000e+09,235000000.0,1.654000e+09,4.162000e+09,...,0.040637,Agilent Technologies Inc,Health Care,1.0,45.89,39.85,38.58,-3.334060,11.317440,14.981856
3,2016-07-31,A,48.11,46.7904,45.6913,-2.348986,2.139000e+09,235000000.0,1.654000e+09,4.162000e+09,...,0.037419,Agilent Technologies Inc,Health Care,3.0,44.36,40.92,40.95,8.453562,17.570870,17.484737
4,2016-08-31,A,46.98,45.6913,45.9121,0.483243,2.139000e+09,235000000.0,1.654000e+09,4.162000e+09,...,0.038333,Agilent Technologies Inc,Health Care,2.0,48.11,45.89,36.31,-2.348784,2.375245,29.385844


In [14]:
# To write this dataframe to an SQL table with any keys, do the following
# Use the "df.to_sql" to write the dataframe to an SQL table with a temporary name
final_table_name = "MasterData_ML"

# Now, write the dataframe to the SQL table
master_df.to_sql('Temp', con=engine, if_exists='replace', index=False)

# Drop any existing NEW table, the one that will contain the data and index
sql_stmt = "DROP TABLE " + final_table_name
engine.execute(sql_stmt)

# Create a "MasterData" table to hold all the data, and be sure to include the index definition
create_MasterData_sql = 'CREATE TABLE "' + final_table_name + '" (' + \
    'monthend_date DATE, ' + \
    'ticker VARCHAR(8), ' + \
    'price FLOAT, ' + \
    'wealth_index FLOAT, ' + \
    'end_wealth_index FLOAT, ' + \
    'total_return FLOAT, ' + \
    'cash FLOAT, ' + \
    'st_debt FLOAT, ' + \
    'lt_debt FLOAT, ' + \
    'equity FLOAT, ' + \
    'revenue FLOAT, ' + \
    'ebit FLOAT, ' + \
    'net_income FLOAT, ' + \
    'basic_eps FLOAT, ' + \
    'diluted_eps FLOAT, ' + \
    'shares FLOAT, ' + \
    'qtr_date DATE, ' + \
    'price_earnings FLOAT, ' + \
    'earnings_yield FLOAT, ' + \
    'book_value FLOAT, ' + \
    'price_book FLOAT, ' + \
    'book_yield FLOAT, ' + \
    'market_cap FLOAT, ' + \
    'net_debt FLOAT, ' + \
    'net_debt_capital FLOAT, ' + \
    'enterprise_value FLOAT, ' + \
    'ev_revenue FLOAT, ' + \
    'revenue_ev FLOAT, ' + \
    'ev_ebit FLOAT, ' + \
    'ebit_ev FLOAT, ' + \
    'name VARCHAR, ' + \
    'sector VARCHAR, ' + \
    'quantile_total_return INTEGER, ' + \
    'price_1_month_ago FLOAT, ' + \
    'price_3_months_ago FLOAT, ' + \
    'price_12_months_ago FLOAT, ' + \
    'trailing_1_month_return FLOAT, ' + \
    'trailing_3_month_return FLOAT, ' + \
    'trailing_12_month_return FLOAT, ' + \
    'PRIMARY KEY (monthend_date, ticker));'

# Create the table.  It will be empty
engine.execute(create_MasterData_sql)

# Now copy the data from the Temp table into the New table
sql_stmt = "INSERT INTO " + final_table_name + " SELECT * FROM Temp"
engine.execute(sql_stmt)

# Delete the Temp table
sql_stmt = "DROP TABLE Temp"
engine.execute(sql_stmt)